In [55]:
import pandas as pd
from collections import Counter               #для подсчёта наиболее встречающихся значений методом most_common, задание 11
from datetime import datetime                 #для работы с датами в задании 21
from itertools import combinations, chain     #для перебора комбинаций пар актёров и создания плоского списка задания 27
import locale                                 #для вывода названий месяцев задания 21 на кириллице (русском языке)
locale.setlocale(locale.LC_ALL, 'ru_RU.UTF-8') 

'ru_RU.UTF-8'

In [2]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
790,tt1210166,50000000,110206216,Moneyball,Brad Pitt|Jonah Hill|Robin Wright|Philip Seymo...,Bennett Miller,What are you really worth?,The story of Oakland Athletics general manager...,133,Drama,Columbia Pictures|Scott Rudin Productions,9/22/2011,6.9,2011
822,tt1540133,6000000,19560274,The Guard,Brendan Gleeson|Don Cheadle|Liam Cunningham|Ma...,John Michael McDonagh,The FBI are about to discover that things work...,An unorthodox Irish policeman with a confronta...,96,Action|Comedy|Thriller|Crime,UK Film Council|Crescendo Productions|Element ...,7/7/2011,6.6,2011
449,tt1126591,55000000,89519773,Burlesque,Cher|Christina Aguilera|Eric Dane|Cam Gigandet...,Steve Antin,It takes a legend... to make a star,The Burlesque Lounge has its best days behind ...,119,Drama|Romance,Bedford Falls Productions,11/23/2010,6.8,2010
1509,tt0469641,63000000,162945894,World Trade Center,Nicolas Cage|Maria Bello|Maggie Gyllenhaal|Mic...,Oliver Stone,The World Saw Evil That Day. Two Men Saw Somet...,"On September, 11th 2001, after the terrorist a...",128,Drama|History|Thriller,Paramount Pictures|Double Feature Films|Interm...,8/9/2006,5.9,2006
826,tt1692486,25000000,27603069,Carnage,Kate Winslet|Jodie Foster|Christoph Waltz|John...,Roman Polanski,A new comedy of no manners,"In Brooklyn Bridge Park, eleven-year-old Zacha...",80,Comedy|Drama,Wild Bunch|Zanagar Films|Canal+|Constantin Fil...,9/16/2011,6.9,2011


In [5]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [3]:
answers = {} #словарь с ответами
data['date'] = pd.to_datetime(data['release_date']) #стобец с преобразованной датой, в формате datetime
data['profit'] = data['revenue'] - data['budget'] #столбец с данными прибыли (сборы минус бюджет)
"""
Создаём столбцы для данных, содержащих неколько значений, преобразованных в списки 
(режиссёры, актёры, кинокомпании)
"""
data['directors'] = data['director'].str.split('|')  
data['actors'] = data['cast'].str.split('|')
data['companies'] = data['production_companies'].str.split('|')

data['title_len'] = data['original_title'].str.len() #столбец с длинами названий фильмов
data['totalwords'] = data['overview'].str.split().str.len() #столбец с количеством слов в описании фильмов

# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [70]:
answers['1'] = '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)'

In [64]:
data.loc[data['budget'] == data['budget'].max(), 'original_title'].values[0]

'Pirates of the Caribbean: On Stranger Tides'

ВАРИАНТ 2

In [36]:
data.sort_values(['budget'], ascending=False).head(1)['original_title'].values[0]

'Pirates of the Caribbean: On Stranger Tides'

# 2. Какой из фильмов самый длительный (в минутах)?

In [71]:
answers['2'] = '2. Gods and Generals (tt0279111)'

In [13]:
data.loc[data['runtime'] == data['runtime'].max(), 'original_title'].values[0]

'Gods and Generals'

# 3. Какой из фильмов самый короткий (в минутах)?





In [72]:
answers['3'] = '3. Winnie the Pooh (tt1449283)'

In [15]:
data.loc[data['runtime'] == data['runtime'].min(), 'original_title'].values[0]

'Winnie the Pooh'

# 4. Какова средняя длительность фильмов?


In [73]:
answers['4'] = '2. 110'

In [82]:
round(data.runtime.mean())

110

# 5. Каково медианное значение длительности фильмов? 

In [74]:
answers['5'] = '1. 107'

In [83]:
round(data.runtime.median())

107

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [75]:
answers['6'] = '5. Avatar (tt0499549)'

In [27]:
data.loc[data['profit'] == data['profit'].max(), 'original_title'].values[0]

'Avatar'

# 7. Какой фильм самый убыточный? 

In [76]:
answers['7'] = '5. The Lone Ranger (tt1210819)'

In [29]:
data.loc[data['profit'] == data['profit'].min(), 'original_title'].values[0]

'The Lone Ranger'

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [77]:
answers['8'] = '1. 1478'

In [32]:
len(data[data['profit'] > 0])

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [78]:
answers['9'] = '4. The Dark Knight (tt0468569)'

In [54]:
data.loc[data['release_year'] == 2008].sort_values(['revenue'], ascending=False).head(1)['original_title'].values[0]

'The Dark Knight'

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [79]:
answers['10'] = '5. The Lone Ranger (tt1210819)'

In [42]:
data.loc[(data['release_year'] >= 2012)&(data['release_year'] <= 2014)].sort_values(['profit']).head(1)['original_title'].values[0]

'The Lone Ranger'

# 11. Какого жанра фильмов больше всего?

In [80]:
answers['11'] = '3. Drama'

In [72]:
pd.Series(data['genres'].str.split('|').sum()).value_counts().index[0]

'Drama'

ВАРИАНТ 2

In [81]:
Counter(data['genres'].str.split('|').sum()).most_common()[0][0]

'Drama'

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [81]:
answers['12'] = '1. Drama'

In [84]:
pd.Series(data[data['profit']>0]['genres'].str.split('|').sum()).value_counts().index[0]

'Drama'

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [82]:
answers['13'] = '5. Peter Jackson'

In [31]:
data.explode('directors').groupby(['directors']).revenue.sum().sort_values(ascending=False).index[0]

'Peter Jackson'

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [83]:
answers['14'] = '3. Robert Rodriguez'

In [54]:
pd.Series(data[data['genres'].str.contains('Action')]['director'].str.split('|').sum()).value_counts().index[0]

'Robert Rodriguez'

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [84]:
answers['15'] = '3. Chris Hemsworth'

In [46]:
data[data['release_year'] == 2012].explode('actors').groupby(['actors']).revenue.sum().sort_values(ascending=False).index[0]

'Chris Hemsworth'

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [85]:
answers['16'] = '3. Matt Damon'

In [52]:
pd.Series(data[data['budget'] > data['budget'].mean()]['cast'].str.split('|').sum()).value_counts().index[0]

'Matt Damon'

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [86]:
answers['17'] = '2. Action'

In [60]:
pd.Series(data[data['cast'].str.contains('Nicolas Cage')]['genres'].str.split('|').sum()).value_counts().index[0]

'Action'

# 18. Самый убыточный фильм от Paramount Pictures

In [87]:
answers['18'] = '1. K-19: The Widowmaker (tt0267626)'

In [87]:
data[data['production_companies'].str.contains('Paramount Pictures')]\
    .groupby(['original_title']).profit.min().sort_values().index[0]

'K-19: The Widowmaker'

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [88]:
answers['19'] = '5. 2015'

In [20]:
data.groupby(['release_year']).revenue.sum().idxmax()

2015

# 20. Какой самый прибыльный год для студии Warner Bros?

In [89]:
answers['20'] = '1. 2014'

In [84]:
data[data['production_companies'].str.contains('Warner Bros')]\
    .groupby(['release_year']).revenue.sum().idxmax()

2014

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [90]:
answers['21'] = '4. Сентябрь'

In [129]:
print(f'''Номер искомого месяца: {data['date'].dt.month.value_counts().index[0]}''')

Номер искомого месяца: 9


In [119]:
data.groupby(data['date'].dt.strftime('%B')).imdb_id.count().sort_values(ascending=False).index[0]

'Сентябрь'

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [91]:
answers['22'] = '2. 450'

In [138]:
len(data[data['date'].dt.month.isin([6, 7, 8])])

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [92]:
answers['23'] = '5. Peter Jackson'

In [13]:
pd.Series(data[data['date'].dt.month.isin([1, 2, 12])]['director'].str.split('|').sum()).value_counts().index[0]

'Peter Jackson'

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [93]:
answers['24'] = '5. Four By Two Productions'

In [47]:
data.explode('companies').groupby(['companies'])['title_len'].mean().sort_values(ascending=False).index[0]

'Four By Two Productions'

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [94]:
answers['25'] = '3. Midnight Picture Show'

In [52]:
data.explode('companies').groupby(['companies'])['totalwords'].mean().sort_values(ascending=False).index[0]

'Midnight Picture Show'

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 


In [95]:
answers['26'] = '1. Inside Out, The Dark Knight, 12 Years a Slave'

In [65]:
data[data['vote_average']>data.vote_average.quantile(0.99)]['original_title'] #про квантиль подсмотрено в Slack)

9                                          Inside Out
34                                               Room
118                                      Interstellar
119                           Guardians of the Galaxy
125                                The Imitation Game
128                                         Gone Girl
138                          The Grand Budapest Hotel
370                                         Inception
599                                   The Dark Knight
872                                       The Pianist
1081    The Lord of the Rings: The Return of the King
1183                          The Wolf of Wall Street
1191                                 12 Years a Slave
1800                                          Memento
Name: original_title, dtype: object

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [96]:
answers['27'] = '5. Daniel Radcliffe & Rupert Grint'

In [62]:
actors_pairs_list = []  
#заполняем список возможными комбинациями пар актёров по каждому фильму
for actor in data.actors:
    actors_pairs_list.append(list(combinations(actor,2))) 
#объединяем полученные списки в общий плоский список пар и ищем количество вхождений по каждой паре актёров
pd.Series(chain.from_iterable(actors_pairs_list)).value_counts().head(5) 

(Daniel Radcliffe, Rupert Grint)       8
(Daniel Radcliffe, Emma Watson)        8
(Rupert Grint, Emma Watson)            7
(Ben Stiller, Owen Wilson)             6
(Johnny Depp, Helena Bonham Carter)    6
dtype: int64

# Submission

In [97]:
# ответы на вопросы
answers

{'1': '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': '2. Gods and Generals (tt0279111)',
 '3': '3. Winnie the Pooh (tt1449283)',
 '4': '2. 110',
 '5': '1. 107',
 '6': '5. Avatar (tt0499549)',
 '7': '5. The Lone Ranger (tt1210819)',
 '8': '1. 1478',
 '9': '4. The Dark Knight (tt0468569)',
 '10': '5. The Lone Ranger (tt1210819)',
 '11': '3. Drama',
 '12': '1. Drama',
 '13': '5. Peter Jackson',
 '14': '3. Robert Rodriguez',
 '15': '3. Chris Hemsworth',
 '16': '3. Matt Damon',
 '17': '2. Action',
 '18': '1. K-19: The Widowmaker (tt0267626)',
 '19': '5. 2015',
 '20': '1. 2014',
 '21': '4. Сентябрь',
 '22': '2. 450',
 '23': '5. Peter Jackson',
 '24': '5. Four By Two Productions',
 '25': '3. Midnight Picture Show',
 '26': '1. Inside Out, The Dark Knight, 12 Years a Slave',
 '27': '5. Daniel Radcliffe & Rupert Grint'}

In [98]:
# общее количество ответов
len(answers)

27